In [1]:
import math 
import re
import itertools

import numpy as np
import pandas as pd
from IPython.display import display, HTML

import sqlite3
import json

In [2]:
# Utility Functions and Classes
def digit_cnt(n):
    if n > 0:
        return int(math.log10(n))+1
    elif n == 0:
        return 1
    elif n < 0:
        return int(math.log10(-n))+2
    

In [3]:
con = sqlite3.connect("/home/skye/Documents/Scripts/FRC/2022/scouting/cblite_lakeview_m52/wildrank.cblite2/db.sqlite3")

In [4]:
cursor = con.cursor()

In [5]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('docs',), ('revs',), ('sqlite_sequence',), ('localdocs',), ('views',), ('info',), ('maps_1',), ('sqlite_stat1',)]


In [6]:
# TBA Retrieved Data
cursor.execute("SELECT json FROM revs where doc_type = 'match' AND doc_id=1")
col_names = list(map(lambda x: x[0], cursor.description))
print(col_names)
print(cursor.fetchall())

df_tba_results = pd.read_sql_query("SELECT json FROM revs where doc_type = 'match'", con)
df_tba_results_e = df_tba_results.join(df_tba_results['json'].apply(json.loads).apply(pd.Series))
df_tba_results_e = df_tba_results_e.sort_values(by=['time'],ignore_index=True).reset_index(drop=True)
#df_tba_results_alliances = pd.DataFrame.from_dict(df_tba_results_e['alliances'])
df_tba_results_alliances = pd.concat({k: pd.DataFrame(v).T for k, v in df_tba_results_e['alliances'].items()}, axis=0)
df_tba_results_alliances = pd.DataFrame(df_tba_results_alliances['team_keys'])

df_tba_results_alliances[['team1','team2','team3']] = pd.DataFrame(df_tba_results_alliances['team_keys'].tolist(), index= df_tba_results_alliances.index)
all_teams = pd.unique(df_tba_results_alliances[['team1','team2','team3']].values.ravel('K'))
#display(HTML(df_tba_results_alliances.to_html()))
#print(all_teams)

team_matches = {}
for team in all_teams:
    select_indices = list(np.where((df_tba_results_alliances["team1"] == team) | \
                                   (df_tba_results_alliances["team2"] == team) | \
                                   (df_tba_results_alliances["team3"] == team))[0])
    team_matches[team] = select_indices
    

#print(pd.DataFrame(team_matches).T)

# we do this list comprehension thing to handle the differnt number of matches a team has if they're a surrogate
df_alliance_color = pd.DataFrame({ key:pd.Series(value) for key, value in team_matches.items() }).T
#df_alliance_color.replace({0: 'red', 1: 'blue'}, inplace=True) 
#display(HTML(df_alliance_color.to_html()))

['json']
[]


In [7]:
# Get our data
cursor.execute("SELECT json FROM revs where doc_type = 'pit'")
df_user_results = pd.read_sql_query("SELECT json FROM revs where doc_type = 'pit'", con)
col_names = list(map(lambda x: x[0], cursor.description))
results = cursor.fetchall()
print(col_names)
pd.options.display.max_colwidth =10000

df_user_results.head(60)



['json']


,json
0,"b'{""data"":{""climb_time"":""About 10sec"",""auto_location"":""Near Edges of Field (2)"",""length"":0.0,""weight"":123.0,""auto_shots_low"":0,""retract_intake"":true,""drivetrain"":""Swerve"",""cargo_scorer"":""Turreted Single Flywheel Short"",""auto_drives"":true,""external_intake"":true,""vision"":true,""auto_shots_high"":5,""width"":0.0,""climb_height"":""2"",""height"":0.0},""type"":""pit"",""team_key"":""frc1188"",""users"":[""1000""]}'"
1,"b'{""data"":{""climb_time"":""About 10sec"",""auto_location"":""No Preference"",""length"":31.0,""weight"":127.0,""auto_shots_low"":1,""retract_intake"":true,""drivetrain"":""Tank w/ Omni corners"",""cargo_scorer"":""Fixed Single Flywheel Short"",""auto_drives"":true,""external_intake"":true,""vision"":false,""auto_shots_high"":0,""width"":27.0,""climb_height"":""2"",""height"":39.0},""type"":""pit"",""team_key"":""frc1940"",""users"":[""1000""]}'"
2,"b'{""data"":{""climb_time"":""About 15sec"",""auto_location"":""Near Edges of Field (2)"",""length"":29.0,""weight"":120.0,""auto_shots_low"":0,""retract_intake"":true,""drivetrain"":""Swerve"",""cargo_scorer"":""Turreted Multi Flywheel Tall"",""auto_drives"":true,""external_intake"":true,""vision"":true,""auto_shots_high"":5,""width"":29.0,""climb_height"":""0"",""height"":6.0},""type"":""pit"",""team_key"":""frc2767"",""users"":[""1000""]}'"
3,"b'{""data"":{""climb_time"":""About 05sec or less"",""auto_location"":""Near Driverstation (1)"",""length"":30.0,""weight"":124.0,""auto_shots_low"":0,""retract_intake"":true,""drivetrain"":""Swerve"",""cargo_scorer"":""Turreted Single Flywheel Tall"",""auto_drives"":true,""external_intake"":true,""vision"":true,""auto_shots_high"":2,""width"":30.0,""climb_height"":""2"",""height"":46.0},""type"":""pit"",""team_key"":""frc2959"",""users"":[""1000""]}'"
4,"b'{""data"":{""climb_time"":""About 15sec"",""auto_location"":""No Preference"",""length"":0.0,""weight"":127.0,""auto_shots_low"":2,""retract_intake"":false,""drivetrain"":""Mecanum"",""cargo_scorer"":""Fixed Multi Flywheel Short"",""auto_drives"":true,""external_intake"":true,""vision"":false,""auto_shots_high"":0,""width"":0.0,""climb_height"":""2"",""height"":0.0},""type"":""pit"",""team_key"":""frc3658"",""users"":[""1000""]}'"
5,"b'{""data"":{""climb_time"":""About 20sec"",""auto_location"":""No Preference"",""length"":27.0,""weight"":115.0,""auto_shots_low"":0,""retract_intake"":false,""drivetrain"":""Swerve"",""cargo_scorer"":""Fixed Single Flywheel Short"",""auto_drives"":true,""external_intake"":false,""vision"":false,""auto_shots_high"":1,""width"":33.0,""climb_height"":""4"",""height"":0.0},""type"":""pit"",""team_key"":""frc4325"",""users"":[""1000""]}'"
6,"b'{""data"":{""climb_time"":""About 10sec"",""auto_location"":""No Preference"",""length"":28.0,""weight"":122.0,""auto_shots_low"":0,""retract_intake"":false,""drivetrain"":""Tank w/ Omni corners"",""cargo_scorer"":""Turreted Single Flywheel Short"",""auto_drives"":false,""external_intake"":true,""vision"":true,""auto_shots_high"":3,""width"":28.0,""climb_height"":""2"",""height"":38.0},""type"":""pit"",""team_key"":""frc4327"",""users"":[""1000""]}'"
7,"b'{""data"":{""climb_time"":""!No Climb"",""auto_location"":""No Preference"",""length"":0.0,""weight"":0.0,""auto_shots_low"":0,""retract_intake"":false,""drivetrain"":""Tank"",""cargo_scorer"":""No Cargo Scoring"",""auto_drives"":true,""external_intake"":false,""vision"":false,""auto_shots_high"":0,""width"":0.0,""climb_height"":""0"",""height"":0.0},""type"":""pit"",""team_key"":""frc4568"",""users"":[""1000""]}'"
8,"b'{""data"":{""climb_time"":""About 20sec"",""auto_location"":""Near Driverstation (1)"",""length"":29.0,""weight"":125.0,""auto_shots_low"":2,""retract_intake"":true,""drivetrain"":""Mecanum"",""cargo_scorer"":""Fixed Single Flywheel Tall"",""auto_drives"":true,""external_intake"":true,""vision"":false,""auto_shots_high"":0,""width"":30.0,""climb_height"":""4"",""height"":46.0},""type"":""pit"",""team_

In [26]:
# convert our column of json data to a df with all variables from that json
df_user_results_e = df_user_results.join(df_user_results['json'].apply(json.loads).apply(pd.Series))
#print(df_user_results_e.info())
print(df_user_results_e['data'].iloc[0])
df_user_results_e = df_user_results_e.join(pd.DataFrame(list(df_user_results_e['data'])))

print(df_user_results_e.columns)


{'climb_time': 'About 10sec', 'auto_location': 'Near Edges of Field (2)', 'length': 0.0, 'weight': 123.0, 'auto_shots_low': 0, 'retract_intake': True, 'drivetrain': 'Swerve', 'cargo_scorer': 'Turreted Single Flywheel Short', 'auto_drives': True, 'external_intake': True, 'vision': True, 'auto_shots_high': 5, 'width': 0.0, 'climb_height': '2', 'height': 0.0}
Index(['json', 'data', 'type', 'team_key', 'users', 'climb_time',
       'auto_location', 'length', 'weight', 'auto_shots_low', 'retract_intake',
       'drivetrain', 'cargo_scorer', 'auto_drives', 'external_intake',
       'vision', 'auto_shots_high', 'width', 'climb_height', 'height'],
      dtype='object')


In [34]:
cols = ['team_key', 'length', 'width', 'height', 'weight', 'drivetrain', 'external_intake',
        'retract_intake', 'cargo_scorer', 'vision', 'auto_location', 'auto_drives', 'auto_shots_high',
        'auto_shots_low', 'climb_height', 'climb_time']
df_user_results_e = df_user_results_e[cols]

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 3)
a = display(df_user_results_e.T)
a;

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30
team_key,frc1188,frc1940,frc2767,frc2959,frc3658,frc4325,frc4327,frc4568,frc5084,frc5182,frc5205,frc5448,frc5523,frc5535,frc5623,frc6002,frc6078,frc6580,frc6753,frc7144,frc7250,frc7491,frc7658,frc7772,frc7811,frc7814,frc8126,frc8286,frc8373,frc8424,frc8608
length,0.0,31.0,29.0,30.0,0.0,27.0,28.0,0.0,29.0,27.0,31.0,0.0,0.0,30.0,0.0,28.0,0.0,0.0,39.0,28.0,32.0,32.0,0.0,0.0,31.0,30.0,27.0,3.0,0.0,26.0,0.0
width,0.0,27.0,29.0,30.0,0.0,33.0,28.0,0.0,30.0,32.0,22.0,0.0,0.0,28.0,0.0,28.0,0.0,50.0,39.0,28.0,26.0,27.0,0.0,0.0,25.0,30.0,32.0,2.0,0.0,30.0,0.0
height,0.0,39.0,6.0,46.0,0.0,0.0,38.0,0.0,46.0,0.0,0.0,52.0,0.0,46.0,0.0,0.0,0.0,0.0,0.0,0.0,49.0,46.0,51.0,0.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0
weight,123.0,127.0,120.0,124.0,127.0,115.0,122.0,0.0,125.0,113.0,123.0,0.0,85.0,124.0,124.0,123.0,0.0,0.0,115.0,61.0,109.0,112.0,110.0,112.0,0.0,83.0,124.0,67.0,107.0,121.0,123.0
drivetrain,Swerve,Tank w/ Omni corners,Swerve,Swerve,Mecanum,Swerve,Tank w/ Omni corners,Tank,Mecanum,Mecanum,Swerve,Tank,Tank w/ Omni corners,Tank w/ Omni corners,Tank w/ Omni corners,Swerve,Mecanum,Tank,Tank,Tank,Tank,Tank,Mecanum,Tank,Tank,Tank,Mecanum,Tank,Tank w/ Omni corners,Tank,Tank
external_intake,True,True,True,True,True,False,True,False,True,False,True,False,True,True,False,True,False,False,True,False,True,True,True,True,True,True,True,False,True,True,True
retract_intake,True,True,True,True,False,False,False,False,True,False,True,False,True,True,False,True,False,False,True,False,True,True,False,True,True,True,True,False,False,True,True
cargo_scorer,Turreted Single Flywheel Short,Fixed Single Flywheel Short,Turreted Multi Flywheel Tall,Turreted Single Flywheel Tall,Fixed Multi Flywheel Short,Fixed Single Flywheel Short,Turreted Single Flywheel Short,No Cargo Scoring,Fixed Single Flywheel Tall,Fixed Single Flywheel Tall,Everybot Arm,Any Other (Add note),Everybot Arm,Everybot Arm,Fixed Multi Flywheel Tall,Turreted Multi Flywheel Short,Fixed Single Flywheel Tall,Fixed Single Flywheel Tall,Fixed Multi Flywheel Tall,No Cargo Scoring,Everybot Arm,Turreted Single Flywheel Tall,Any Other (Add note),Everybot Arm,Everybot Arm,Fixed Single Flywheel Short,Turreted Single Flywheel Short,No Cargo Scoring,Fixed Multi Flywheel Short,Everybot Arm,Everybot Arm
vision,True,False,True,True,False,False,True,False,False,False,False,False,False,False,True,True,False,True,False,False,False,True,False,False,False,False,True,False,False,False,False


In [37]:
# Get our notes data
cursor.execute("SELECT json FROM revs where doc_type = 'notes'")
df_user_results_notes = pd.read_sql_query("SELECT json FROM revs where doc_type = 'notes'", con)
col_names = list(map(lambda x: x[0], cursor.description))
results = cursor.fetchall()
print(col_names)
pd.options.display.max_colwidth =10000

df_user_results_notes.head(60)



['json']


,json
0,"b'{""notes"":[""auton where steel opponents balls\\n"",""Can shoot while having minor hitting. "",""Held 3 cargo in auton. (on accident)"",""really good shooter but very weak climber\\n"",""didn\'t climb. very fast accurate shots only missed 1"",""hit incapacitated robot."",""good with counter d"",""accurate shooting, quick bot, bumper fell off in"",""\\n"",""held 3 in auto"",""broke."",""has a very accurate shooter and nice swerve drive""],""type"":""notes"",""team_key"":""frc1188"",""users"":[""1000""]}'"
1,"b'{""notes"":[""\\""solid defense\\"". willing to fill"",""Aggresive ramming defense."",""intake fell apart during match\\nclimb mechanism failed "",""can\'t shoot, only defense\\n"",""Does not play effective defense"",""played defense""],""type"":""notes"",""team_key"":""frc1940"",""users"":[""1000""]}'"
2,"b'{""notes"":[""yello"",""sexy robot"",""Such a hot robot"",""the best team"",""Intake belt came off."",""Randomly transformed itself (like bumblebee from \\""Transformers\\"") into a humanoid creature and started beating up the refs, still did traversal"",""our intake got stuck in blue bot"",""we are cool \\uD83D\\uDE04"",""Fact of the day\\n\\nCole is the hottest master scouter you could possible have.\\n"",""hook jumped off""],""type"":""notes"",""team_key"":""frc2767"",""users"":[""1000""]}'"
3,"b'{""notes"":[""Very slow low batter shot. Magazine has issues some times, but doesn\'t jam."",""Intake sometimes lets balls squeeze through sections"",""can not keep cargo. didn\'t happen, but if I had to guess defense on them and they\'d fall out."",""Cole: app closed before I could save. NEGATE THIS FOR TABLEAU!!! \\nCory: lost cargo easily. if they had defense they likely couldn\'t keep many."",""NEGATE match 15\\n"",""consistent lower hub scorer"",""Keeps putting the robot on the field backwards for autonomous"",""Can score well no obstacles (other robots)"",""intook 3 balls, didn\'t fix problem"",""I accidentally restarted mid teleop. they scored more points than counted"",""has an incredibley slow moveable hood.""],""type"":""notes"",""team_key"":""frc2959"",""users"":[""1000""]}'"
4,"b'{""notes"":[""Jammed intake/broken"",""shoots good often made their shots. climbs to the third bar easily.\\n"",""intake gets stuck on balls often (caused it to break completely before)"",""couldn\'t shoot the balls high enough so they kept on bouncing out"",""only shoots low goal. has a very slow intake."",""defense"",""intake does not work properly gets balls stuck\\n""],""type"":""notes"",""team_key"":""frc3658"",""users"":[""1000""]}'"
5,"b'{""notes"":[""robot turned off or just stopped moving\\n"",""could climb higher but something msy have messed"",""could climbhiher"",""could climb higher but also instantly falls upon robot deactivation "",""was not a participate in the match"",""got hit right before shooting causing the ball to get stuck for the rest of the match\\nExtended their climb outside of the hanger"",""Intake just \\""popped\\"" out randomly\\nhave a two ball auton that is MUCH more consistent"",""ball got stuck\\ncould have gotten traversal but not enough room to climb\\n"",""no show in the match"",""climb didn\'t make it"",""mostly single ball fender cycles.""],""type"":""notes"",""team_key"":""frc4325"",""users"":[""1000""]}'"
6,"b'{""notes"":[""2 shots in auto, but looked like wrong auto used."",""Attempts batter shots"",""goes to fast into the climb (under time pressure)"",""has a very small intake and an upper shooter so it missed a lot and had a hard time getting balls"",""intake broke"",""lifted off ground then lowered down"",""Good defense. Could push 5205 around well."",""attempted 5 ball auto"",""defense""],""type"":""notes"",""team_key"":""frc4327"",""users"":[""1000""]}'"
7,"b'{""notes"":[""Quick,"",""defense bot, no shooting and"",""defense bot, no shooting and no climb"",""Doesn\'t do much on defense. Confused."",""tried to play defense but is to light to push anyone"",""no shooter or intake 

In [47]:
# convert our column of json data to a df with all variables from that json
df_user_results_notes_e = df_user_results_notes.join(df_user_results_notes['json'].apply(json.loads).apply(pd.Series))
#print(df_user_results_e.info())
#print(df_user_results_notes_e['notes'].iloc[0])
df_user_results_notes_e = df_user_results_notes_e.join(pd.DataFrame(list(df_user_results_notes_e['notes'])))

print(df_user_results_notes_e[['team_key','notes']])


   team_key                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        notes                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
0   frc1188                            

In [9]:
goal_to_analyze = 1 #0 = unknown, 1 = high, 2=low
color_right_side_app_UI = "blue"        #"red" or "blue" 



#create a filter if we want to exclude any teams from our global stats to prevent dilution
teams_with_data = df_user_results_e['team_key'].unique()
teams_to_exclude = ['frc0000']
teams_to_include = [x for x in teams_with_data if x not in teams_to_exclude]
print(f'Teams used for calculation of averages: {teams_to_include}\n----')
df_user_results_teams = df_user_results_e[df_user_results_e['team_key'].isin(teams_to_include)]


## columns we will use
loc_col_list = ['location_1', 'location_2', 'location_3', 'location_4', 'location_5',
                       'location_6', 'location_7', 'location_8']
cycle_col_list = ['hub_upper_score','hub_lower_score','hub_upper_miss','hub_lower_miss',
                       'location_1', 'location_2', 'location_3', 'location_4', 'location_5',
                       'location_6', 'location_7', 'location_8']

match_col_list = ['post_match-climb_time', 'post_match-climb_attempt',
                  'auto-startLoc','auto-score_cargo_high', 'auto-score_cargo_low','auto-drive',
                  'post_match-tipped', 'auto-miss_cargo_high', 'auto-miss_cargo_low','auto-robot_contact',
                  'post_match-climb_level', 'post_match-climb_fall']
additional_match_col_list = ['alliance_color', 'hub_unknown_scored', 'hub_upper_scored', 'hub_lower_scored',
                            'hub_unknown_missed', 'hub_upper_missed', 'hub_lower_missed']


#Grab stats for all locations from teams querried
all_cycle_locs = pd.DataFrame()
all_match_info = pd.DataFrame()
for idx, d in df_user_results_teams.iterrows():
    match = int(''.join(map(str,re.findall('^.*qm([0-9]+)$',d['match_key'])))) #strip the match number from the end of the match key, the only gaurentee we have is the matches are orginized by the left most digit, easier to just snag the match number and link it to an alliance color 
    team = d['team_key']
    df_tba_match = df_tba_results_alliances.loc[match-1]    #fix out 1 indexed match keys to 0 indexed match data frame
    alliance_color = df_tba_match.loc[(df_tba_match['team1'] == team) | \
                                  (df_tba_match['team2'] == team) | \
                                  (df_tba_match['team3'] == team)].index[0]
  
    m = d['data']
    df_match = pd.DataFrame(m)
    print(type(d['data']))
    if ('cycles' in m) == False :
        print(f"No cycles to parse!")
        continue
    
    df_cycles = pd.DataFrame(m['cycles'])
    if df_cycles.empty:                                               # we need to contitnue and jump this iteration of the for loop if there are no cycles
        print(f"No cycles to parse!")
        continue
    #cycle_col_list = list(df_cycles.columns)
    print('Type', type(df_cycles.columns.values.tolist()))
    print('Type', type(match_col_list[0]))
    df_cycles.replace({False: 0, True: 1}, inplace=True)                # cast boolean values across all columns to int

    #df_cycles = df_cycles[cycle_col_list]                               # TODO df_cycles is out of order without this line, hard to read
    print(df_cycles.info())
    df_match['team_key'] = team[3:]
    df_cycles['team_key'] = team[3:]
    df_match['match_idx'] = str(match)
    df_cycles['match_idx'] = str(match)
    
    
    # build some checksums to ensure internal consistency
    df_cycles['loc'] = df_cycles.loc[:, loc_col_list[::-1]].astype(str).sum(axis=1).astype(int).apply(digit_cnt)  # we invert the list w/ [::-1] because we want 1 to be on the left, not 8
    df_cycles['loc_checksum'] = df_cycles[loc_col_list].sum(axis=1)
    df_cycles.loc[df_cycles['loc_checksum'] != 1, 'loc'] = 0
    
    df_cycles['target_goal'] = df_cycles[cycle_col_list[:4][::-1]].astype(str).sum(axis=1).astype(int).apply(digit_cnt) #0 is unknown, 1 is upper hub goal, 2 is lower hub goal
    df_cycles['missed_shot'] = df_cycles['target_goal'].map({1:0, 2:0, 3:1, 4:1}).fillna(1)    # map target goal vals based on a dict to create the missed shot column
    df_cycles['target_goal'] = df_cycles['target_goal'].map({1:1, 2:2, 3:1, 4:2}).fillna(0)    # map target goal to true target, regardless of make or miss
    df_cycles['target_checksum'] = df_cycles[loc_col_list].sum(axis=1)
    df_cycles.loc[df_cycles['target_checksum'] != 1, 'target_goal'] = 0
    
    df_cycles['defended_shot'] = 0 #temporary, as we are not collecting defended shots, this will trickle down and create a bunch of extra json output, but it is easier vs pulling all the defense code out
                                   #this key would need to be added back into cycle column list in order to be used again
    
    loc_swap_dict = {1:4 , 2:3 , 3:2 , 4:1 , 5:8 , 6:7 , 7:6 , 8:5 }  # alignment mapping of locs to rotate 180deg
    if alliance_color == color_right_side_app_UI:
        df_cycles.replace({"loc":loc_swap_dict}, inplace=True)  
    
    
    all_cycle_locs = all_cycle_locs.append(df_cycles[['team_key', 'match_idx' ,'loc','target_goal','missed_shot','defended_shot']])
    df_match['alliance_color'] = alliance_color
    df_match['hub_unknown_scored'] = ((df_cycles['target_goal'] == 0)&(df_cycles['missed_shot'] == 0)).sum() # this doesn't make sense, we can't conclude a shot made it if we only have location
    df_match['hub_upper_scored'] = ((df_cycles['target_goal'] == 1)&(df_cycles['missed_shot'] == 0)).sum()
    df_match['hub_lower_scored'] = ((df_cycles['target_goal'] == 2)&(df_cycles['missed_shot'] == 0)).sum()
    df_match['hub_unknown_missed'] = ((df_cycles['target_goal'] == 0)&(df_cycles['missed_shot'] == 1)).sum()  # this doesn't make sense, we can't conclude a shot made it if we only have location
    df_match['hub_upper_missed'] = ((df_cycles['target_goal'] == 1)&(df_cycles['missed_shot'] == 1)).sum()
    df_match['hub_lower_missed'] = ((df_cycles['target_goal'] == 2)&(df_cycles['missed_shot'] == 1)).sum()
    
    _match_col_list_tmp = list(itertools.chain(['team_key', 'match_idx'], additional_match_col_list, match_col_list))    #list of cols of non cycle match vars to sent to df that will be ultimatly parsed to json
    df_match_to_all = df_match[_match_col_list_tmp]
    all_match_info = all_match_info.append(df_match_to_all.iloc[:1], ignore_index=True) # we extract a range of [:1] because assigning a single row dataframe (when extracted as .iloc[0]) is viewed as a series, which messes up dtypes
#display(HTML(all_cycle_locs.to_html()))
display(HTML(all_match_info.to_html()))
    

Teams used for calculation of averages: ['frc5067', 'frc5205', 'frc5523', 'frc7144', 'frc8126', 'frc8373', 'frc2767', 'frc2959', 'frc6002', 'frc6753', 'frc7658', 'frc7811', 'frc3658', 'frc4325', 'frc4327', 'frc6078', 'frc6580', 'frc8286', 'frc4568', 'frc5182', 'frc7250', 'frc7772', 'frc8424', 'frc5084', 'frc5448', 'frc5623', 'frc5535', 'frc7209', 'frc7491', 'frc1188', 'frc1940', 'frc7814', 'frc8608']
----
<class 'dict'>
Type <class 'list'>
Type <class 'str'>
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   location_1       5 non-null      int64
 1   hub_upper_score  5 non-null      int64
 2   location_8       5 non-null      int64
 3   location_6       5 non-null      int64
 4   location_7       5 non-null      int64
 5   location_4       5 non-null      int64
 6   location_5       5 non-null      int64
 7   hub_lower_score  5 non-null      in

<class 'dict'>
No cycles to parse!
<class 'dict'>
Type <class 'list'>
Type <class 'str'>
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   location_1       2 non-null      int64
 1   hub_upper_score  2 non-null      int64
 2   location_8       2 non-null      int64
 3   location_6       2 non-null      int64
 4   location_7       2 non-null      int64
 5   location_4       2 non-null      int64
 6   location_5       2 non-null      int64
 7   hub_lower_score  2 non-null      int64
 8   hub_lower_miss   2 non-null      int64
 9   location_2       2 non-null      int64
 10  hub_upper_miss   2 non-null      int64
 11  location_3       2 non-null      int64
dtypes: int64(12)
memory usage: 320.0 bytes
None
<class 'dict'>
No cycles to parse!
<class 'dict'>
Type <class 'list'>
Type <class 'str'>
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entr

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   location_1       12 non-null     int64
 1   hub_upper_score  12 non-null     int64
 2   location_8       12 non-null     int64
 3   location_6       12 non-null     int64
 4   location_7       12 non-null     int64
 5   location_4       12 non-null     int64
 6   location_5       12 non-null     int64
 7   hub_lower_score  12 non-null     int64
 8   hub_lower_miss   12 non-null     int64
 9   location_2       12 non-null     int64
 10  hub_upper_miss   12 non-null     int64
 11  location_3       12 non-null     int64
dtypes: int64(12)
memory usage: 1.2 KB
None
<class 'dict'>
Type <class 'list'>
Type <class 'str'>
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  

<class 'dict'>
No cycles to parse!
<class 'dict'>
Type <class 'list'>
Type <class 'str'>
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   location_1       1 non-null      int64
 1   hub_upper_score  1 non-null      int64
 2   location_8       1 non-null      int64
 3   location_6       1 non-null      int64
 4   location_7       1 non-null      int64
 5   location_4       1 non-null      int64
 6   location_5       1 non-null      int64
 7   hub_lower_score  1 non-null      int64
 8   hub_lower_miss   1 non-null      int64
 9   location_2       1 non-null      int64
 10  hub_upper_miss   1 non-null      int64
 11  location_3       1 non-null      int64
dtypes: int64(12)
memory usage: 224.0 bytes
None
<class 'dict'>
No cycles to parse!
<class 'dict'>
No cycles to parse!
<class 'dict'>
Type <class 'list'>
Type <class 'str'>
<class 'pandas.core.f

<class 'dict'>
No cycles to parse!
<class 'dict'>
Type <class 'list'>
Type <class 'str'>
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   location_1       9 non-null      int64
 1   hub_upper_score  9 non-null      int64
 2   location_8       9 non-null      int64
 3   location_6       9 non-null      int64
 4   location_7       9 non-null      int64
 5   location_4       9 non-null      int64
 6   location_5       9 non-null      int64
 7   hub_lower_score  9 non-null      int64
 8   hub_lower_miss   9 non-null      int64
 9   location_2       9 non-null      int64
 10  hub_upper_miss   9 non-null      int64
 11  location_3       9 non-null      int64
dtypes: int64(12)
memory usage: 992.0 bytes
None
<class 'dict'>
No cycles to parse!
<class 'dict'>
Type <class 'list'>
Type <class 'str'>
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entr

...

<class 'dict'>
No cycles to parse!
<class 'dict'>
No cycles to parse!
<class 'dict'>
Type <class 'list'>
Type <class 'str'>
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   location_1       9 non-null      int64
 1   hub_upper_score  9 non-null      int64
 2   location_8       9 non-null      int64
 3   location_6       9 non-null      int64
 4   location_7       9 non-null      int64
 5   location_4       9 non-null      int64
 6   location_5       9 non-null      int64
 7   hub_lower_score  9 non-null      int64
 8   hub_lower_miss   9 non-null      int64
 9   location_2       9 non-null      int64
 10  hub_upper_miss   9 non-null      int64
 11  location_3       9 non-null      int64
dtypes: int64(12)
memory usage: 992.0 bytes
None
<class 'dict'>
Type <class 'list'>
Type <class 'str'>
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entr

<class 'dict'>
Type <class 'list'>
Type <class 'str'>
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   location_1       9 non-null      int64
 1   hub_upper_score  9 non-null      int64
 2   location_8       9 non-null      int64
 3   location_6       9 non-null      int64
 4   location_7       9 non-null      int64
 5   location_4       9 non-null      int64
 6   location_5       9 non-null      int64
 7   hub_lower_score  9 non-null      int64
 8   hub_lower_miss   9 non-null      int64
 9   location_2       9 non-null      int64
 10  hub_upper_miss   9 non-null      int64
 11  location_3       9 non-null      int64
dtypes: int64(12)
memory usage: 992.0 bytes
None
<class 'dict'>
Type <class 'list'>
Type <class 'str'>
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 12 columns):
 #   Column           Non

<class 'dict'>
No cycles to parse!
<class 'dict'>
Type <class 'list'>
Type <class 'str'>
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   location_1       4 non-null      int64
 1   hub_upper_score  4 non-null      int64
 2   location_8       4 non-null      int64
 3   location_6       4 non-null      int64
 4   location_7       4 non-null      int64
 5   location_4       4 non-null      int64
 6   location_5       4 non-null      int64
 7   hub_lower_score  4 non-null      int64
 8   hub_lower_miss   4 non-null      int64
 9   location_2       4 non-null      int64
 10  hub_upper_miss   4 non-null      int64
 11  location_3       4 non-null      int64
dtypes: int64(12)
memory usage: 512.0 bytes
None
<class 'dict'>
Type <class 'list'>
Type <class 'str'>
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 1

<class 'dict'>
Type <class 'list'>
Type <class 'str'>
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   location_1       6 non-null      int64
 1   hub_upper_score  6 non-null      int64
 2   location_8       6 non-null      int64
 3   location_6       6 non-null      int64
 4   location_7       6 non-null      int64
 5   location_4       6 non-null      int64
 6   location_5       6 non-null      int64
 7   hub_lower_score  6 non-null      int64
 8   hub_lower_miss   6 non-null      int64
 9   location_2       6 non-null      int64
 10  hub_upper_miss   6 non-null      int64
 11  location_3       6 non-null      int64
dtypes: int64(12)
memory usage: 704.0 bytes
None
<class 'dict'>
Type <class 'list'>
Type <class 'str'>
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 12 columns):
 #   Column           No

,team_key,match_idx,alliance_color,hub_unknown_scored,hub_upper_scored,hub_lower_scored,hub_unknown_missed,hub_upper_missed,hub_lower_missed,post_match-climb_time,post_match-climb_attempt,auto-startLoc,auto-score_cargo_high,auto-score_cargo_low,auto-drive,post_match-tipped,auto-miss_cargo_high,auto-miss_cargo_low,auto-robot_contact,post_match-climb_level,post_match-climb_fall
0,5067,10,blue,0,0,4,0,0,1,About 10sec,0,1,0,0,True,False,0,1,False,2,False
1,5205,10,red,2,2,0,0,0,0,About 15sec,2,2,2,0,False,False,0,0,False,2,False
2,5523,10,red,0,0,1,0,0,0,!No Climb,0,1,0,0,False,True,0,0,False,0,False
3,8126,10,red,0,1,0,0,4,0,About 10sec,2,1,0,0,True,False,0,0,False,2,False
4,8373,10,blue,0,13,0,0,1,0,!No Climb,0,2,1,0,True,False,1,0,False,0,False
5,2767,11,red,0,11,0,0,3,0,About 20sec,4,2,2,0,True,False,1,0,False,4,False
6,2959,11,blue,0,4,4,0,0,0,About 05sec or less,1,1,1,0,True,False,0,0,False,2,False
7,6002,11,blue,0,1,0,0,4,0,About 30sec +,4,2,0,0,True,False,1,0,False,4,False
8,6753,11,blue,0,3,0,0,3,0,!No Climb,3,1,0,0,True,False,0,0,False,3,False
9,7658,11,red,0,0,4,0,0,0,About 15sec,4,1,0,0,True,False,0,1,False,4,False


In [10]:
#generate a dummy col for every possible location target miss and defense combination
loc_range = list(range(0,9,1))
target_range = list(range(0,3,1))
missed_range = [0,1]
defense_range = [0,1]
team_dummy = "0000"
match_idx_dummy = "0"
cycles_dummy = pd.DataFrame(itertools.product(loc_range, target_range, missed_range, defense_range),
                            columns=['loc', 'target_goal', 'missed_shot', 'defended_shot'])
cycles_dummy['team_key'] = team_dummy
cycles_dummy['match_idx'] = match_idx_dummy

agg_cycles = all_cycle_locs
agg_cycles = pd.concat([agg_cycles, cycles_dummy], axis=0, ignore_index=True)
#display(HTML(agg_cycles.sort_values(by=['team_key', 'match_idx']).to_html()))

agg_cycles['tgt-loc-miss-def'] = (agg_cycles['target_goal']).astype(str) \
                                + (agg_cycles['loc']).astype(str) \
                                + (agg_cycles['missed_shot']).astype(str) \
                                + (agg_cycles['defended_shot']).astype(str)

agg_cycles = pd.get_dummies(agg_cycles, prefix=['tgt-loc-miss-def'], columns=["tgt-loc-miss-def"])
agg_cycles.drop(['loc', 'target_goal', 'missed_shot', 'defended_shot'], axis=1, inplace=True)
agg_cycles_col_names = agg_cycles.columns
agg_cycles = agg_cycles.groupby(['team_key', 'match_idx'],as_index=False)[agg_cycles_col_names[2:]].sum()


# <--- 
agg_cycles_col_names = agg_cycles.columns

hub_scored_upper_clean = agg_cycles.groupby(['team_key', 'match_idx'])[agg_cycles_col_names[42:71:4]].sum()
hub_scored_lower_clean = agg_cycles.groupby(['team_key', 'match_idx'])[agg_cycles_col_names[78:116:4]].sum()
hub_missed_upper_clean = agg_cycles.groupby(['team_key', 'match_idx'])[agg_cycles_col_names[44:73:4]].sum()
hub_missed_lower_clean = agg_cycles.groupby(['team_key', 'match_idx'])[agg_cycles_col_names[80:118:4]].sum()

hub_scored_upper_def = agg_cycles.groupby(['team_key', 'match_idx'])[agg_cycles_col_names[43:72:4]].sum()
hub_scored_lower_def = agg_cycles.groupby(['team_key', 'match_idx'])[agg_cycles_col_names[79:117:4]].sum()
hub_missed_upper_def = agg_cycles.groupby(['team_key', 'match_idx'])[agg_cycles_col_names[45:74:4]].sum()
hub_missed_lower_def = agg_cycles.groupby(['team_key', 'match_idx'])[agg_cycles_col_names[81:119:4]].sum()

shots_clean = pd.DataFrame()
shots_def = pd.DataFrame()
shots_all = pd.DataFrame()
for loc in range(1,9): 
    df_clean = pd.DataFrame({'location_clean': loc,
                                'hub_scored_upper_clean':hub_scored_upper_clean.iloc[:,loc-1],
                                'hub_scored_lower_clean':hub_scored_lower_clean.iloc[:,loc-1],
                                'hub_missed_upper_clean':hub_missed_upper_clean.iloc[:,loc-1],
                                'hub_missed_lower_clean':hub_missed_lower_clean.iloc[:,loc-1]})
    shots_clean[f'shots_clean{loc}'] = df_clean.apply(pd.Series.to_dict, axis=1)
    
    df_def = pd.DataFrame({'location_def': loc,
                                'hub_scored_upper_def':hub_scored_upper_def.iloc[:,loc-1],
                                'hub_scored_lower_def':hub_scored_lower_def.iloc[:,loc-1],
                                'hub_missed_upper_def':hub_missed_upper_def.iloc[:,loc-1],
                                'hub_missed_lower_def':hub_missed_lower_def.iloc[:,loc-1]})
    shots_def[f'shots_def{loc}'] = df_def.apply(pd.Series.to_dict, axis=1)
    
    df_all = pd.merge(df_clean, df_def, how='left', left_on=['team_key','match_idx'], right_on = ['team_key','match_idx'])
    df_all['location_all'] = df_all['location_clean']
    df_all['hub_scored_upper_all'] = df_all['hub_scored_upper_clean'] + df_all['hub_scored_upper_def']
    df_all['hub_scored_lower_all'] = df_all['hub_scored_lower_clean'] + df_all['hub_scored_lower_def']
    df_all['hub_missed_upper_all'] = df_all['hub_missed_upper_clean'] + df_all['hub_missed_upper_def']
    df_all['hub_missed_lower_all'] = df_all['hub_missed_lower_clean'] + df_all['hub_missed_lower_def']
    df_all = df_all[['location_all','hub_scored_upper_all', 'hub_scored_lower_all','hub_missed_upper_all', 'hub_missed_lower_all']]
    shots_all[f'shots_all{loc}'] = df_all.apply(pd.Series.to_dict, axis=1)
    
print(shots_def.head())
print(shots_all.head())
shots_clean['shots_clean'] = shots_clean.apply(pd.Series.to_list, axis=1)
shots_clean = shots_clean[['shots_clean']].reset_index() 

shots_def['shots_def'] = shots_def.apply(pd.Series.to_list, axis=1)
shots_def = shots_def[['shots_def']].reset_index()

shots_all['shots_all'] = shots_all.apply(pd.Series.to_list, axis=1)
shots_all = shots_all[['shots_all']].reset_index()

# --->

# --- Clean up some climb data stuff
# assumption is: all climb levels will be utilized at some point, or else we need to create some matches with different climb levels for dummy team 0000
all_match_info = pd.concat([all_match_info, pd.get_dummies(all_match_info['post_match-climb_level'], prefix='post_match-climb_level')], axis=1)
all_match_info = pd.concat([all_match_info, pd.get_dummies(all_match_info['post_match-climb_attempt'], prefix='post_match-climb_attempt')], axis=1)
all_match_info.drop(['post_match-climb_level','post_match-climb_attempt'], axis=1, inplace=True)
# --->

# --- Combine everything to send to json
agg_cycles = pd.merge(agg_cycles, shots_clean, how='left', left_on=['team_key','match_idx'], right_on = ['team_key','match_idx'])
agg_cycles = pd.merge(agg_cycles, shots_def, how='left', left_on=['team_key','match_idx'], right_on = ['team_key','match_idx'])
agg_cycles = pd.merge(agg_cycles, shots_all, how='left', left_on=['team_key','match_idx'], right_on = ['team_key','match_idx'])
agg_cycles = agg_cycles[agg_cycles.columns.drop(list(agg_cycles.filter(regex='tgt-loc-miss-def')))]
agg_cycles = agg_cycles.drop(agg_cycles[(agg_cycles['team_key'] == "0000") & \
                                        (agg_cycles['match_idx'] == "0")].index)
all_match_info['post_match-climb_time'].replace({"!No Climb": 0, "About 05sec or less": 5, "About 10sec":10, "About 15sec":15, \
                                                 "About 20sec":20, "About 25sec":25, "About 30sec +":30, }, inplace=True)

team_matches = pd.merge(all_match_info, agg_cycles,  how='left', left_on=['team_key','match_idx'], right_on = ['team_key','match_idx'])
display(HTML(team_matches.sort_values(by=['team_key', 'match_idx']).to_html()))

                                                                                                                                         shots_def1  \
team_key match_idx                                                                                                                                    
0000     0          {'location_def': 1, 'hub_scored_upper_def': 1, 'hub_scored_lower_def': 1, 'hub_missed_upper_def': 1, 'hub_missed_lower_def': 1}   
1188     1          {'location_def': 1, 'hub_scored_upper_def': 0, 'hub_scored_lower_def': 0, 'hub_missed_upper_def': 0, 'hub_missed_lower_def': 0}   
         16         {'location_def': 1, 'hub_scored_upper_def': 0, 'hub_scored_lower_def': 0, 'hub_missed_upper_def': 0, 'hub_missed_lower_def': 0}   
         22         {'location_def': 1, 'hub_scored_upper_def': 0, 'hub_scored_lower_def': 0, 'hub_missed_upper_def': 0, 'hub_missed_lower_def': 0}   
         28         {'location_def': 1, 'hub_scored_upper_def': 0, 'hub_scored_lower_def': 0, 

,team_key,match_idx,alliance_color,hub_unknown_scored,hub_upper_scored,hub_lower_scored,hub_unknown_missed,hub_upper_missed,hub_lower_missed,post_match-climb_time,auto-startLoc,auto-score_cargo_high,auto-score_cargo_low,auto-drive,post_match-tipped,auto-miss_cargo_high,auto-miss_cargo_low,auto-robot_contact,post_match-climb_fall,post_match-climb_level_0,post_match-climb_level_1,post_match-climb_level_2,post_match-climb_level_3,post_match-climb_level_4,post_match-climb_attempt_0,post_match-climb_attempt_1,post_match-climb_attempt_2,post_match-climb_attempt_3,post_match-climb_attempt_4,shots_clean,shots_def,shots_all
44,1188,1,blue,0,7,0,0,1,0,15,2,3,0,True,False,0,0,False,False,0,0,1,0,0,1,0,0,0,0,"[{'location_clean': 1, 'hub_scored_upper_clean': 0, 'hub_scored_lower_clean': 0, 'hub_missed_upper_clean': 0, 'hub_missed_lower_clean': 0}, {'location_clean': 2, 'hub_scored_upper_clean': 0, 'hub_scored_lower_clean': 0, 'hub_missed_upper_clean': 0, 'hub_missed_lower_clean': 0}, {'location_clean': 3, 'hub_scored_upper_clean': 0, 'hub_scored_lower_clean': 0, 'hub_missed_upper_clean': 0, 'hub_missed_lower_clean': 0}, {'location_clean': 4, 'hub_scored_upper_clean': 0, 'hub_scored_lower_clean': 0, 'hub_missed_upper_clean': 0, 'hub_missed_lower_clean': 0}, {'location_clean': 5, 'hub_scored_upper_clean': 7, 'hub_scored_lower_clean': 0, 'hub_missed_upper_clean': 1, 'hub_missed_lower_clean': 0}, {'location_clean': 6, 'hub_scored_upper_clean': 0, 'hub_scored_lower_clean': 0, 'hub_missed_upper_clean': 0, 'hub_missed_lower_clean': 0}, {'location_clean': 7, 'hub_scored_upper_clean': 0, 'hub_scored_lower_clean': 0, 'hub_missed_upper_clean': 0, 'hub_missed_lower_clean': 0}, {'location_clean': 8, 'hub_scored_upper_clean': 0, 'hub_scored_lower_clean': 0, 'hub_missed_upper_clean': 0, 'hub_missed_lower_clean': 0}]","[{'location_def': 1, 'hub_scored_upper_def': 0, 'hub_scored_lower_def': 0, 'hub_missed_upper_def': 0, 'hub_missed_lower_def': 0}, {'location_def': 2, 'hub_scored_upper_def': 0, 'hub_scored_lower_def': 0, 'hub_missed_upper_def': 0, 'hub_missed_lower_def': 0}, {'location_def': 3, 'hub_scored_upper_def': 0, 'hub_scored_lower_def': 0, 'hub_missed_upper_def': 0, 'hub_missed_lower_def': 0}, {'location_def': 4, 'hub_scored_upper_def': 0, 'hub_scored_lower_def': 0, 'hub_missed_upper_def': 0, 'hub_missed_lower_def': 0}, {'location_def': 5, 'hub_scored_upper_def': 0, 'hub_scored_lower_def': 0, 'hub_missed_upper_def': 0, 'hub_missed_lower_def': 0}, {'location_def': 6, 'hub_scored_upper_def': 0, 'hub_scored_lower_def': 0, 'hub_missed_upper_def': 0, 'hub_missed_lower_def': 0}, {'location_def': 7, 'hub_scored_upper_def': 0, 'hub_scored_lower_def': 0, 'hub_missed_upper_def': 0, 'hub_missed_lower_def': 0}, {'location_def': 8, 'hub_scored_upper_def': 0, 'hub_scored_lower_def': 0, 'hub_missed_upper_def': 0, 'hub_missed_lower_def': 0}]","[{'location_all': 1, 'hub_scored_upper_all': 0, 'hub_scored_lower_all': 0, 'hub_missed_upper_all': 0, 'hub_missed_lower_all': 0}, {'location_all': 2, 'hub_scored_upper_all': 0, 'hub_scored_lower_all': 0, 'hub_missed_upper_all': 0, 'hub_missed_lower_all': 0}, {'location_all': 3, 'hub_scored_upper_all': 0, 'hub_scored_lower_all': 0, 'hub_missed_upper_all': 0, 'hub_missed_lower_all': 0}, {'location_all': 4, 'hub_scored_upper_all': 0, 'hub_scored_lower_all': 0, 'hub_missed_upper_all': 0, 'hub_missed_lower_all': 0}, {'location_all': 5, 'hub_scored_upper_all': 7, 'hub_scored_lower_all': 0, 'hub_missed_upper_all': 1, 'hub_missed_lower_all': 0}, {'location_all': 6, 'hub_scored_upper_all': 0, 'hub_scored_lower_all': 0, 'hub_missed_upper_all': 0, 'hub_missed_lower_all': 0}, {'location_all': 7, 'hub_scored_upper_all': 0, 'hub_scored_lower_all': 0, 'hub_missed_upper_all': 0, 'hub_missed_lower_all': 0}, {'location_all': 8, 'hub_scored_upper_all': 0, 'hub_scored_lower_all': 0, 'hub_missed_upper_all': 0, 'hub_missed_lower_all': 0}]"
28,1188,16,blue,0,10,0,0,1,0,0,2,3,0,True,False,1,0,False,False,1,0,0,0,0,1,0,0,0,0,"[{'location_clean': 1, '

In [11]:
result = team_matches.to_json(orient="records")

parsed = json.loads(result)

x = json.dumps(parsed, indent=4) 
x = '{"data":' + x +'}'
print(x)

out_path = "./out.json"
with open(out_path, 'w') as f:
    f.write(x)

{"data":[
    {
        "team_key": "5067",
        "match_idx": "10",
        "alliance_color": "blue",
        "hub_unknown_scored": 0,
        "hub_upper_scored": 0,
        "hub_lower_scored": 4,
        "hub_unknown_missed": 0,
        "hub_upper_missed": 0,
        "hub_lower_missed": 1,
        "post_match-climb_time": 10,
        "auto-startLoc": "1",
        "auto-score_cargo_high": 0,
        "auto-score_cargo_low": 0,
        "auto-drive": true,
        "post_match-tipped": false,
        "auto-miss_cargo_high": 0,
        "auto-miss_cargo_low": 1,
        "auto-robot_contact": false,
        "post_match-climb_fall": false,
        "post_match-climb_level_0": 0,
        "post_match-climb_level_1": 0,
        "post_match-climb_level_2": 1,
        "post_match-climb_level_3": 0,
        "post_match-climb_level_4": 0,
        "post_match-climb_attempt_0": 1,
        "post_match-climb_attempt_1": 0,
        "post_match-climb_attempt_2": 0,
        "post_match-climb_attempt_3": 0